In [41]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error

def compute_metrics(img1, img2):
    data_range = img2.max() - img2.min()
    s, _ = ssim(img1, img2, data_range=data_range, full=True)
    p = psnr(img1, img2, data_range=data_range)
    m = mean_squared_error(img1.flatten(), img2.flatten())
    return s, p, m

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Could not load {path}")
    return img

if __name__ == "__main__":
    base_gt_dir    = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/GroundTruth_Merged"
    base_pred_dir  = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Denoised_Angle_Merged_Images"
    base_input_dir = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Input_Merged"

    organs = {
        "membrane": [171, 172, 175, 176, 179],
        "nucleus":  [169, 170, 173, 174, 177]
    }

    for organelle, ids in organs.items():
        print(f"\n=== {organelle.upper()} ===")
        for idx in ids:
            prefix = f"image_{idx}_{organelle}"
            gt_path   = os.path.join(base_gt_dir, f"{prefix}_fused_MIP.tif")
            pred_path = os.path.join(base_pred_dir, organelle.capitalize(), f"{prefix}_denoised.tif")
            inp_path  = os.path.join(base_input_dir, f"{prefix}_angle_MIP.tif")

            # load and resize if needed
            gt   = load_image(gt_path)
            pred = load_image(pred_path)
            inp  = load_image(inp_path)
            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)
            if inp.shape != gt.shape:
                inp  = cv2.resize(inp,  (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            # Compute SSIMs
            pred_s, pred_p, pred_m = compute_metrics(pred, gt)
            ref_s,  ref_p,  ref_m  = compute_metrics(inp, gt)
            norm_s = (pred_s - ref_s) / (1 - ref_s) if ref_s < 1 else float('nan')

            print(
    f"{prefix}: "
    f"ref_ssim={ref_s:.4f}, ref_psnr={ref_p:.2f}, ref_mse={ref_m:.2f} | "
    f"pred_ssim={pred_s:.4f}, pred_psnr={pred_p:.2f}, pred_mse={pred_m:.2f} | "
    f"normalized_ssim={norm_s:.4f}"
)



=== MEMBRANE ===
image_171_membrane: ref_ssim=0.7993, ref_psnr=24.44, ref_mse=22396.11 | pred_ssim=0.8001, pred_psnr=24.27, pred_mse=22731.40 | normalized_ssim=0.0039
image_172_membrane: ref_ssim=0.9087, ref_psnr=28.90, ref_mse=24635.47 | pred_ssim=0.9234, pred_psnr=31.26, pred_mse=24049.38 | normalized_ssim=0.1613
image_175_membrane: ref_ssim=0.5819, ref_psnr=19.39, ref_mse=31154.32 | pred_ssim=0.5878, pred_psnr=19.26, pred_mse=31058.59 | normalized_ssim=0.0142
image_176_membrane: ref_ssim=0.7146, ref_psnr=21.34, ref_mse=31956.74 | pred_ssim=0.7294, pred_psnr=22.73, pred_mse=31595.26 | normalized_ssim=0.0519
image_179_membrane: ref_ssim=0.7557, ref_psnr=22.64, ref_mse=26305.56 | pred_ssim=0.7528, pred_psnr=22.49, pred_mse=27897.12 | normalized_ssim=-0.0117

=== NUCLEUS ===
image_169_nucleus: ref_ssim=0.8740, ref_psnr=24.62, ref_mse=26222.06 | pred_ssim=0.8717, pred_psnr=25.91, pred_mse=29414.05 | normalized_ssim=-0.0186
image_170_nucleus: ref_ssim=0.6936, ref_psnr=22.29, ref_mse=3021

In [45]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error

def compute_metrics(img1, img2):
    data_range = img2.max() - img2.min()
    s, _ = ssim(img1, img2, data_range=data_range, full=True)
    p = psnr(img1, img2, data_range=data_range)
    m = mean_squared_error(img1.flatten(), img2.flatten())
    return s, p, m

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Could not load {path}")
    return img

if __name__ == "__main__":
    base_gt_dir    = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/GroundTruth_Merged"
    base_pred_dir  = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Denoised_Angle_Merged_Images"
    base_input_dir = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Input_Merged"

    organs = {
        "membrane": [171, 172, 175, 176, 179],
        "nucleus":  [169, 170, 173, 174, 177]
    }

    for organelle, ids in organs.items():
        # Print organelle title
        print(f"\n=== {organelle.upper()} ===")
        # Print table header
        header = (
            f"{'Image':<25}"
            f"{'Ref SSIM':>10}{'Ref PSNR':>10}{'Ref MSE':>10}"
            f"{'Pred SSIM':>12}{'Pred PSNR':>12}{'Pred MSE':>12}"
            f"{'Norm SSIM':>12}"
        )
        print(header)
        print("-" * len(header))

        for idx in ids:
            prefix = f"image_{idx}_{organelle}"
            gt_path   = os.path.join(base_gt_dir, f"{prefix}_fused_MIP.tif")
            pred_path = os.path.join(base_pred_dir, organelle.capitalize(), f"{prefix}_denoised.tif")
            inp_path  = os.path.join(base_input_dir, f"{prefix}_angle_MIP.tif")

            # load and resize if needed
            gt   = load_image(gt_path)
            pred = load_image(pred_path)
            inp  = load_image(inp_path)
            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)
            if inp.shape != gt.shape:
                inp  = cv2.resize(inp,  (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            # Compute metrics
            pred_s, pred_p, pred_m = compute_metrics(pred, gt)
            ref_s,  ref_p,  ref_m  = compute_metrics(inp, gt)
            norm_s = (pred_s - ref_s) / (1 - ref_s) if ref_s < 1 else float('nan')

            # Print one row of the table
            print(
                f"{prefix:<25}"
                f"{ref_s:10.4f}{ref_p:10.2f}{ref_m:10.2f}"
                f"{pred_s:12.4f}{pred_p:12.2f}{pred_m:12.2f}"
                f"{norm_s:12.4f}"
            )


=== MEMBRANE ===
Image                      Ref SSIM  Ref PSNR   Ref MSE   Pred SSIM   Pred PSNR    Pred MSE   Norm SSIM
-------------------------------------------------------------------------------------------------------
image_171_membrane           0.7993     24.44  22396.11      0.8001       24.27    22731.40      0.0039
image_172_membrane           0.9087     28.90  24635.47      0.9234       31.26    24049.38      0.1613
image_175_membrane           0.5819     19.39  31154.32      0.5878       19.26    31058.59      0.0142
image_176_membrane           0.7146     21.34  31956.74      0.7294       22.73    31595.26      0.0519
image_179_membrane           0.7557     22.64  26305.56      0.7528       22.49    27897.12     -0.0117

=== NUCLEUS ===
Image                      Ref SSIM  Ref PSNR   Ref MSE   Pred SSIM   Pred PSNR    Pred MSE   Norm SSIM
-------------------------------------------------------------------------------------------------------
image_169_nucleus            